# LAB 9

Networks and Systems Security
Week 07
Gen AI Security



## Aims of the Seminar

This laboratory exercise introduces you to fundamental security
considerations in generative artificial intelligence. Students will deploy a
local large language model using Ollama and investigate key
vulnerabilities, threat categories, and defensive strategies highlighted in
the lecture material. The aim is to develop awareness of risks inherent in
generative systems and to practise core security evaluation procedures
in a controlled environment.

Learning Objectives:
- Demonstrate the ability to run a local LLM instance and evaluate
its behaviour under controlled security-testing scenarios.
- Identify major generative AI threat vectors, including prompt
injection, data poisoning, inversion, model theft, and adversarial
manipulation.
- Apply basic red-teaming principles to assess the robustness and
safety of a deployed model.
- Recommend high-level mitigation strategies based on security
best practices for model deployment, governance, and incident
response.


### Setting up

In [ ]:
!pip install ollama

# !ollama -version
# Since it didnt work i installed it directly from the docs instead

Search all available models on Ollama website via: https://ollama.com/search 
(Chose llava: https://ollama.com/library/llava)

Review Local LLMs leadership dashboard on hugging Face 🤗:  https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard#/?params=0%2C8&official=true


Part I — Deploying a Local Model with Ollama

In [2]:
# !ollama pull smollm2:135m
# Since it didnt work i installed it directly from the docs instead

!ollama -v

ollama version is 0.13.1


In [ ]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='llava', messages=[
    {
        'role': 'user',
        'content': 'Why is the sky blue?',
    },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)
